In [1]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request
from quantumnet.objects import Request
import asyncio

rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 5
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

Hosts inicializados
Canais inicializados
Pares EPRs adicionados


In [2]:
# Parte adicional para simulação com timeslot usando programação assíncrona
async def process_request(request, rede, controlador, current_time, execution_delay=3):
    """
    Processa uma única request após um delay de registro (3 time-slots).
    Se a regra já existe, executa diretamente.
    """
    request.starttime = current_time
    alice = rede.get_host(request.alice)
    rule = alice.find_rule_by_request(request)
    # Caso não exista um match na tabela
    if rule is False:
        print(f"[Time {current_time}] Registrando nova regra para Request {request}.")
        controlador.add_match_route_rule_in_host_reactive(request)
        await asyncio.sleep(execution_delay)  # Delay para registro
        print(f"[Time {current_time + execution_delay}] Regra registrada para {request}, executando:")
        rule = alice.find_rule_by_request(request)
        controlador.run_rule(rule[1])
        request.endtime = current_time + execution_delay
    # Caso a regra já exista
    else:
        await asyncio.sleep(1) 
        print(f"[Time {current_time}] Executando regra para Request {request}:")
        controlador.run_rule(rule[1])
        request.endtime = current_time + 1
    print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

async def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com timeslots contínuos usando programação assíncrona.
    - As requests são processadas simultaneamente.
    - O tempo avança de forma contínua enquanto as requisições são executadas.
    """
    current_time = 0  # Tempo global inicial
    tasks = []  # Lista de tarefas assíncronas para execução das requests

    while requests or tasks:
        # Adicionar nova request ao processamento
        if requests:
            request = requests.pop(0)
            print(f"[Time {current_time}] Request {request} recebida.")
            task = asyncio.create_task(process_request(request, rede, controlador, current_time))
            tasks.append(task)

        # Aguarda 1 time-slot antes de avançar o tempo
        await asyncio.sleep(1)
        current_time += 1

        # Filtrar tarefas concluídas
        tasks = [task for task in tasks if not task.done()]

    print("Simulação concluída!")

# Uso da função
# asyncio.run(simulate_requests_with_timeslots(rede, requests, controlador))
r1 = Request(0, 1, 0.5, 2)
r2 = Request(0, 1, 0.5, 2)
r3 = Request(0, 1, 0.5, 2)
r4 = Request(0, 1, 0.5, 2)
requests2 = [r1, r2, r3, r4]
await simulate_requests_with_timeslots(rede, requests2, controlador)

[Time 0] Request R-55B1D0 recebida.
[Time 0] Registrando nova regra para Request R-55B1D0.
Decisão aplicada: (Normal E2E,)
[Time 1] Request R-018D10 recebida.
[Time 2] Request R-5D69C0 recebida.
[Time 1] Executando regra para Request R-018D10:
Passo: 1
Request R-018D10: Start Time = 1, End Time = 2
[Time 3] Regra registrada para R-55B1D0, executando:
Passo: 1
Request R-55B1D0: Start Time = 0, End Time = 3
[Time 3] Request R-E22150 recebida.
[Time 2] Executando regra para Request R-5D69C0:
Passo: 1
Request R-5D69C0: Start Time = 2, End Time = 3
[Time 3] Executando regra para Request R-E22150:
Passo: 1
Request R-E22150: Start Time = 3, End Time = 4
Simulação concluída!


In [3]:
# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com a introdução de timeslots.
    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.
    """
    current_time = 0  # Tempo global da simulação
    for request in requests:
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = current_time
            current_time += 3  # Delay de 3 time-slots para registrar nova regra
            controlador.add_match_route_rule_in_host_reactive(request)
            request.endtime = current_time
        else:  # Caso já exista a regra
            request.starttime = current_time
            current_time += 1  # Delay de 1 time-slot para execução
            controlador.run_rule(rule[1])
            request.endtime = current_time
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

In [4]:
simulate_requests_with_timeslots(rede, requests, controlador)

Decisão aplicada: (High Fidelity,)
Request R-A842F0: Start Time = 0, End Time = 3
Decisão aplicada: (High Fidelity,)
Request R-57E490: Start Time = 3, End Time = 6
Decisão aplicada: (Normal E2E,)
Request R-57E5D0: Start Time = 6, End Time = 9
Decisão aplicada: (Normal E2E,)
Request R-576B10: Start Time = 9, End Time = 12
Decisão aplicada: (Normal E2E,)
Request R-576C40: Start Time = 12, End Time = 15


In [5]:
# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com timeslots contínuos.
    - O tempo passa de 1 em 1 independentemente da requisição.
    - Nova regra leva 3 time-slots para ser registrada.
    - As regras são executadas no momento correto assim que o tempo alcança o necessário.
    """
    current_time = 0  # Tempo global da simulação
    pending_requests = []  # Lista de requisições pendentes para registro
    
    while requests or pending_requests:
        # Processar requests pendentes para execução
        for pending in pending_requests[:]:
            if pending['start_time'] + 3 == current_time:  # Após 3 time-slots
                controlador.add_match_route_rule_in_host_reactive(pending['request'])
                pending['request'].endtime = current_time
                print(f"Request {pending['request']}: Start Time = {pending['request'].starttime}, End Time = {pending['request'].endtime}")
                pending_requests.remove(pending)
        
        # Processar nova requisição apenas se ainda há requests
        if requests:
            request = requests.pop(0)
            alice = rede.get_host(request.alice)
            rule = alice.find_rule_by_request(request)
            
            if rule == False:  # Caso não exista um match na tabela
                request.starttime = current_time
                pending_requests.append({'request': request, 'start_time': current_time})
            else:  # Caso já exista a regra
                request.starttime = current_time
                controlador.run_rule(rule[1])
                request.endtime = current_time + 1
                print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")
        
        # Incrementar o tempo global
        current_time += 1

        # Log do tempo atual
        print(f"Time Slot: {current_time}")